# Example flow for processing and aggregating stats about committee meeting attendees and protocol parts

See the [DataFlows documentation](https://github.com/datahq/dataflows) for more details regarding the Flow object and processing functions.

Feel free to modify and commit changes which demonstrate additional functionality or relevant data.

## Constants

In [ ]:
# Limit processing of protocol parts for development
PROCESS_PARTS_LIMIT = 2

# Enable caching of protocol parts data (not efficient, should only be used for local development with sensible PROCESS_PARTS_LIMIT)
PROCESS_PARTS_CACHE = True

# Filter the meetings to be processed, these kwargs are passed along to DataFlows filter_rows processor for meetings resource
MEETINGS_FILTER_ROWS_KWARGS = {'equals': [{'KnessetNum': 20}]}

# Don'e use local data - loads everything from knesset data remote storage
# When set to False - also enables caching, so you won't download from remote storage on 2nd run.
USE_DATA = False

## Loading lexicons

In [51]:
import os

def read_topic_to_set(topic_name):
    lines = open(os.path.join(dir_name, topic_name + ".txt"), 'r').readlines()
    return set([line.strip().replace("\ufeff", "") for line in lines])

dir_name = "../topics/lexicons"        

files = os.listdir(dir_name)

topics = [file.split('.')[0] for file in files]

lexicons = {}
for topic_name in topics:
    lexicons[topic_name] = read_topic_to_set(topic_name)

## Load source data

In [4]:
from dataflows import filter_rows, cache
from datapackage_pipelines_knesset.common_flow import load_knesset_data, load_member_names

# Loads a dict containing mapping between knesset member id and the member name
member_names = load_member_names(use_data=USE_DATA)

# define flow steps for loading the source committee meetings data
# the actual loading is done later in the Flow
load_steps = (
    load_knesset_data('people/committees/meeting-attendees/datapackage.json', USE_DATA),
    filter_rows(**MEETINGS_FILTER_ROWS_KWARGS)
)

if not USE_DATA:
    # when loading from URL - enable caching which will skip loading on 2nd run
    load_steps = (cache(*load_steps, cache_path='.cache/people-committee-meeting-attendees-knesset-20'),)

loading from url: https://storage.googleapis.com/knesset-data-pipelines/data/members/mk_individual/datapackage.json
using cache data from .cache/members-mk-individual-names
loading from url: https://storage.googleapis.com/knesset-data-pipelines/data/people/committees/meeting-attendees/datapackage.json


## Inspect the datapackages which will be loaded

Last command's output log should contain urls to datapackage.json files, open them and check the table schema to see the resource metadata and available fields which you can use in the processing functions.

Check the [frictionlessdata docs](https://frictionlessdata.io/docs/) for more details about the datapackage file format.

## Extract topics from lines

In [107]:
from collections import defaultdict
from dataflows import Flow
import pandas as pd
import numpy as np

first = True
running_index = 0

meeting_data_global = None
topics_df = None

stats = defaultdict(int)
member_attended_meetings = defaultdict(int)
rows = []
m_rows = []

def initialize_meeting_data_global(meeting_row):
    global meeting_data_global
    global running_index
    topics_exists = meeting_row['topics'] is not None
    topics = ";".join(meeting_row['topics']) if topics_exists else ""

    meeting_data_global = {
        'KnessetNum': [meeting_row['KnessetNum']],
        'CommitteeSessionID': [meeting_row['CommitteeSessionID']],
        'Number': [meeting_row['Number']],
        'Mks': [';'.join(meeting_row['mks'])] if meeting_row['mks'] is not None else [""],
        'Topics': [topics],
        'StartDate': [meeting_row['StartDate']],
        'CommitteeID': [meeting_row['CommitteeID']]
    } 
    
    # Adding topic counts in the 'topics' column 
    if(topics_exists):
        topic_words = topics.split()
        topic_words_size_2 = [" ".join(topic_words[i:i+2]) for i in range(len(topic_words) - 2)]
        topic_words_size_3 = [" ".join(topic_words[i:i+3]) for i in range(len(topic_words) - 3)]
      
    for topic_name, lexicon in lexicons.items():
        if not topics_exists:
            meeting_data_global[topic_name + "_score"] = [0]
        else:
            count = lexicon_count(lexicon, topic_words) + lexicon_count(lexicon, topic_words_size_2) + lexicon_count(lexicon, topic_words_size_3)
            meeting_data_global[topic_name + "_score"] = [count*3]
    
def word_permutations(word):
    clean_word = word.strip()
    permutations = [clean_word]
    if len(word) > 1 and word.startswith('ה') or word.startswith('ב') or word.startswith('ל'):
        permutations.append(word[1:])
    return permutations


def in_lexicon(word, lexicon):
    for p in word_permutations(word):
        if p in lexicon:
            return True
    return False

            
def lexicon_count(lexicon, words):
    count = 0
    for word in words:
        if in_lexicon(word, lexicon):
            count += 1
    return count   


def process_meeting_protocol_part(row):
    global meeting_data_global
    global running_index
    global rows
    rows.append(row)
    stats['processed parts'] += 1
    words = row['header'].split() if row['header'] is not None else []
    words += row['body'].split() if row['body'] is not None else []
    words_size_2 = [" ".join(words[i:i+2]) for i in range(len(words) - 2)]
    words_size_3 = [" ".join(words[i:i+3]) for i in range(len(words) - 3)]
                    
    for topic_name, lexicon in lexicons.items():
        meeting_data_global[topic_name + "_score"][0] += lexicon_count(lexicon, words)
        meeting_data_global[topic_name + "_score"][0] += lexicon_count(lexicon, words_size_2)  
        meeting_data_global[topic_name + "_score"][0] += lexicon_count(lexicon, words_size_3)  
            

def process_meeting(row):
    global topics_df
    global running_index
    
    m_rows.append(row)
    stats['total meetings'] += 1
    if row['attended_mk_individual_ids']:
        for mk_id in row['attended_mk_individual_ids']:
            member_attended_meetings[mk_id] += 1
    parts_filename = row['parts_parsed_filename']
    if parts_filename:
        initialize_meeting_data_global(row)
        if running_index < 2 and PROCESS_PARTS_LIMIT and stats['processed parts'] < PROCESS_PARTS_LIMIT:
            steps = (load_knesset_data('committees/meeting_protocols_parts/' + parts_filename, USE_DATA),)
            if not USE_DATA and PROCESS_PARTS_CACHE:
                steps = (cache(*steps, cache_path='.cache/committee-meeting-protocol-parts/' + parts_filename),)
            steps += (process_meeting_protocol_part,)
            Flow(*steps).process()
            running_index += 1

def add_to_table(row):
    global topics_df
    global meeting_data_global
    if topics_df is None:
        topics_df = pd.DataFrame(meeting_data_global)
    else:
        topics_df = topics_df.append(pd.DataFrame(meeting_data_global), ignore_index=True)


process_steps = (
    process_meeting,
    add_to_table,
)

In [108]:
from dataflows import Flow, dump_to_path

Flow(*load_steps, *process_steps, dump_to_path('data/committee-meeting-attendees-parts')).process()

using cache data from .cache/people-committee-meeting-attendees-knesset-20
loading from url: https://storage.googleapis.com/knesset-data-pipelines/data/committees/meeting_protocols_parts/files/5/6/562716.csv
using cache data from .cache/committee-meeting-protocol-parts/files/5/6/562716.csv


(<datapackage.package.Package at 0x7f6434f6ef98>,
 {'count_of_rows': 10256,
  'bytes': 30129277,
  'hash': '9bb63d3b4c724c88df1416113d0fb80c',
  'dataset_name': None})

In [110]:
topics_df[:100]

,KnessetNum,CommitteeSessionID,Number,Mks,Topics,StartDate,CommitteeID,Diplomacy_score,Ecologics_score,Economics_score,Education_score,Health_score,Security_score
0,20,562716,1,"זאב אלקין – יו""ר הוועדה המסדרת;ניסן סלומינסקי ...",בחירת יושב-ראש לוועדת הכספים הזמנית,2015-04-01 14:00:00,922,0,0,11,0,0,0
1,20,563293,2,"ניסן סלומינסקי – היו""ר;אלי אלאלוף;יצחק כהן;אלי...","אישור השקעת המדינה בחברת דואר ישראל בע""מ בהתאם...",2015-04-16 10:30:00,922,0,0,0,0,0,0
2,20,563590,3,"ניסן סלומינסקי – היו""ר;אלי אלאלוף;יצחק כהן;אלי...","אישור השקעת המדינה בחברת דואר ישראל בע""מ בהתאם...",2015-04-16 12:30:00,922,0,0,0,0,0,0
3,20,563523,2,"זאב אלקין – היו""ר;מיקי זוהר;נאוה בוקר;יואב בן ...",החלטה בדבר כינוס המליאה בפגרה לצורך מינוי סגני...,2015-04-15 09:45:00,938,0,0,0,0,0,0
4,20,562190,1,"זאב אלקין – היו""ר;יריב לוין;דני דנון;רוברט איל...",אישור מועדי פגרת הפסח.;בחירת ועדות זמניות לעני...,2015-03-31 18:30:00,938,0,0,3,0,0,0
5,20,564118,4,"ניסן סלומינסקי – היו""ר;יצחק כהן;מיקי לוי;אורי ...","הארכת מועדים לפי חוק מימון מפלגות, התשל""ג-1973",2015-05-11 13:00:00,922,0,0,0,0,0,0
6,20,564120,5,"ניסן סלומינסקי – היו""ר;אלי אלאלוף;יצחק כהן;מיק...",אישור מוסדות ציבור לעניין סעיף 46 לפקודת מס הכ...,2015-05-11 13:20:00,922,0,0,0,0,0,0
7,20,564217,3,"זאב אלקין – היו""ר;רוברט אילטוב;נאוה בוקר;דוד ב...",הצעת חוק-יסוד: הממשלה (תיקון מס' 3 והוראת שעה ...,2015-05-11 10:15:00,938,0,0,0,0,0,0
8,20,564304,4,"זאב אלקין – היו""ר;רוברט אילטוב;נאוה בוקר;דוד ב...",הצעת חוק-יסוד: הממשלה (תיקון מס' 3 והוראת שעה ...,2015-05-11 13:50:00,938,0,0,0,0,0,0
9,20,564337,6,"זאב אלקין – היו""ר;רוברט אילטוב;נאוה בוקר;דוד ב...",הצעת חוק-יסוד: הממשלה (תיקון מס' 3 והוראת שעה ...,2015-05-12 00:40:00,938,0,0,0,0,0,0


In [112]:
with open("topics_output.csv", 'w') as f:
    f.write(topics_df.to_csv())

In [ ]:
topics_df[]

## Aggregate and print stats

In [113]:
from collections import deque
import yaml

top_attended_member_names = [member_names[mk_id] for mk_id, num_attended in
                             deque(sorted(member_attended_meetings.items(), key=lambda kv: kv[1]), maxlen=5)]
print('\n')
print('-- top attended members --')
print(top_attended_member_names)
print('\n')
print('-- stats --')
print(yaml.dump(dict(stats), default_flow_style=False, allow_unicode=True))



-- top attended members --
['איתן ברושי', 'לוי מיקי', 'דוב חנין', 'רפאל משה גפני', 'אורי מקלב']


-- stats --
processed parts: 42
total meetings: 10256



## Get output data

Output data is available in the left sidebar under data directory, you can check the datapackage.json and created csv file to explore the data and schema.